In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

Se importa el archivo de datos llamado imputer.csv. Y se muestra el df. En este tiene 2 columnas A y B. Como informacion adicional se sabe que si A=0 entonces B=0.

In [10]:
df = pd.read_csv('imputer.csv')
df

,A,B
0,1,NaN
1,3,20.0
2,0,0.0
3,4,30.0
4,3,25.0
5,2,NaN
6,2,15.0
7,0,0.0
8,3,20.0
9,4,30.0


Se cuentan los NaNs. (6)

In [4]:
df.isna().sum()

A    0
B    6
dtype: int64

Entonces se tiene un data frame con dos columnas A y B.   
A no tiene NaNs pero B tiene 6 NaNs.  
  
Se quiere reemplazar los NaNs de B por la media de B sin considerar los A,B=0.  
  
1) Se hace "a mano" 
2) Se hace con imputer y se verifica si coincide con lo hecho "a mano"

1) Se copia el df y se obtiene la media de B.

In [11]:
dfm=df.copy()
AB0=dfm['A']!=0             # Condicion para tener en cuenta en el promedio a los B cuyos A = 0.
b_media=dfm[AB0]['B'].mean()
b_media

23.75

Se reemplaza en B y se muestra como deberia quedar el Data Frame.

In [12]:
dfm.B=dfm.B.fillna(b_media)
dfm

,A,B
0,1,23.75
1,3,20.00
2,0,0.00
3,4,30.00
4,3,25.00
5,2,23.75
6,2,15.00
7,0,0.00
8,3,20.00
9,4,30.00


Ahora se hace lo mismo con imputer, para ello primero se hace otra copia de df.   
Y se verifica el dfi coincida con el original. 

In [34]:
dfi=df.copy()
dfi

,A,B
0,1,NaN
1,3,20.0
2,0,0.0
3,4,30.0
4,3,25.0
5,2,NaN
6,2,15.0
7,0,0.0
8,3,20.0
9,4,30.0


Se instancia imputer, eliguiendo la estrategia mean.   
Y además se ajusta o entrena ("fit") con la columana B sin tener en cuenta los A=0.

In [35]:
imputer = SimpleImputer(strategy='mean')
imputer.fit(dfi[AB0][['B']])    
# Notar que fit acepta como entrada SOLO tipo dataframe por eso se coloca dfi[AB0][['B']] y no dfi[AB0]['B'] que devolveria una serie.
# Otra opcion seria entrar con dfi.loc[AB0,['B']]
# O pd.DataFrame(dfi[AB0]['B'])

SimpleImputer()

Se comprueba que va a utilizar la media para reemplazar los NaNs.

In [36]:
imputer.statistics_


array([23.75])

Se aplica imputer a la columna B y se verifica el resultado comparandolo con lo hecho en el punto 1.

In [37]:
dfi.loc[AB0,['B']] = imputer.transform(dfi[AB0][['B']])
# Aca tambien notar que transform solo acepta entrada como dataframe.df[['B']] NO df['B']
# Notar que en el lado izquierdo de la igualdad se debe utilizar loc sino da muchos errores de referencias y/o advertencias.
# Ademas de ambos lados del igual se debe tener un dataframe de la misma dimension.
dfi

,A,B
0,1,23.75
1,3,20.00
2,0,0.00
3,4,30.00
4,3,25.00
5,2,23.75
6,2,15.00
7,0,0.00
8,3,20.00
9,4,30.00


Tambien se podria haber utilizado imputer.fit_transform(df) todo en uno.

In [28]:
dft=df.copy()
dft

,A,B
0,1,NaN
1,3,20.0
2,0,0.0
3,4,30.0
4,3,25.0
5,2,NaN
6,2,15.0
7,0,0.0
8,3,20.0
9,4,30.0


In [33]:
imputer2 = SimpleImputer(strategy='mean')
dft.loc[AB0,['B']] = imputer2.fit_transform(dft.loc[AB0,['B']])
dft

,A,B
0,1,23.75
1,3,20.00
2,0,0.00
3,4,30.00
4,3,25.00
5,2,23.75
6,2,15.00
7,0,0.00
8,3,20.00
9,4,30.00


Por último Notar que el registro 12 en donde A=0 y B=NaN, el cual no fue logicamente reemplazado ya que debe ser reemplazado por 0.

In [38]:
dfi.loc[dfi['A']==0,['B']]=0
dfm.loc[dfm['A']==0,['B']]=0
dft.loc[dft['A']==0,['B']]=0

dfi

,A,B
0,1,23.75
1,3,20.00
2,0,0.00
3,4,30.00
4,3,25.00
5,2,23.75
6,2,15.00
7,0,0.00
8,3,20.00
9,4,30.00
